# Custom Chatbot Project

TODO: In this cell, write an explanation of which dataset you have chosen and why it is appropriate for this task
I chose the wikipedia article "2024" because i thought it would be interesting to see how much the chat bot is able to learn about the most recent year

## Data Wrangling

TODO: In the cells below, load your chosen dataset into a `pandas` dataframe with a column named `"text"`. This column should contain all of your text data, separated into at least 20 rows.

In [4]:
import requests
response= requests.get("https://en.wikipedia.org/w/api.php?action=query&prop=extracts&exlimit=1&titles=2024&explaintext=1&formatversion=2&format=json")
response

<Response [200]>

In [6]:
response.json()["query"]["pages"][0]["extract"].split("\n")

['2024 (MMXXIV) was a leap year starting on Monday of the Gregorian calendar, the 2024th year of the Common Era (CE) and Anno Domini (AD) designations, the 24th  year of the 3rd millennium and the 21st century, and the  5th   year of the 2020s decade.  ',
 "The year saw the continuation of major armed conflicts, including the Russian invasion of Ukraine, the Myanmar civil war, the Sudanese civil war, and the Islamist insurgency in the Sahel. Israel's war on Gaza led to widespread protests and spillover conflicts into numerous other countries, most notably Lebanon, which was invaded by Israel in October. This followed an intensification of the ongoing conflict between Israel and Hezbollah. In September, Israel escalated an offensive against the group, which resulted in the killing of the Secretary-General, Hassan Nasrallah. Ismail Haniyeh, the political leader of Hamas, had also been assassinated in the Iranian capital Tehran in July, and his successor Yahya Sinwar was killed by the Isr

In [7]:
import pandas as pd
df = pd.DataFrame()
df["text"] = response.json()["query"]["pages"][0]["extract"].split("\n")
df = df[df["text"].str.len() >0]
df = df[~df["text"].str.startswith("==")]
df.reset_index(inplace=True, drop=True)
df

,text
0,2024 (MMXXIV) was a leap year starting on Mond...
1,The year saw the continuation of major armed c...
2,"Approximately 80 countries, representing aroun..."
3,January 1
4,"Egypt, Ethiopia, Iran and the United Arab Emir..."
...,...
266,"Economics – Daron Acemoglu, Simon Johnson and ..."
267,"Literature – Han Kang, for her intense poetic ..."
268,"Peace – Nihon Hidankyo, for its efforts to ach..."
269,Physics – John J. Hopfield and Geoffrey E. Hin...


## Custom Query Completion

TODO: In the cells below, compose a custom query using your chosen dataset and retrieve results from an OpenAI `Completion` model. You may copy and paste any useful code from the course materials.

In [15]:
import openai
openai.api_base = "https://openai.vocareum.com/v1"
openai.api_key = "your-key"
EMBEDDING_MODEL_NAME = "text-embedding-ada-002"

response = openai.Embedding.create(
    input=df["text"].tolist(),
    model=EMBEDDING_MODEL_NAME
)

type(response)
response.keys()
type(response["data"])
response["data"][0]
lenresponse["data"][0]["embedding"]
embeddings= [ data["embedding"] for data in response["data"]]
embeddings
df["embeddings"] = embeddings
df

InvalidRequestError: Insufficient budget available

In [11]:
import tiktoken

def create_prompt(question, df, max_token_count):
    """
    Given a question and a dataframe containing rows of text and their
    embeddings, return a text prompt to send to a Completion model
    """
    tokenizer = tiktoken.get_encoding("cl100k_base")

    prompt_template = """
Answer the question based on the context below, and if the question
can't be answered based on the context, say "I don't know"

Context: 

{}

---

Question: {}
Answer:"""

    current_token_count = len(tokenizer.encode(prompt_template)) + \
                            len(tokenizer.encode(question))

    context = []
    for text in get_rows_sorted_by_relevance(question, df)["text"].values:

        # Increase the counter based on the number of tokens in this row
        text_token_count = len(tokenizer.encode(text))
        current_token_count += text_token_count

        # Add the row of text to the list if we haven't exceeded the max
        if current_token_count <= max_token_count:
            context.append(text)
        else:
            break

    return prompt_template.format("\n\n###\n\n".join(context), question)

In [12]:
COMPLETION_MODEL_NAME = "gpt-3.5-turbo-instruct"

def answer_question(
    question, df, max_prompt_tokens=1800, max_answer_tokens=150
):
    """
    Given a question, a dataframe containing rows of text, and a maximum
    number of desired tokens in the prompt and response, return the
    answer to the question according to an OpenAI Completion model

    If the model produces an error, return an empty string
    """

    prompt = create_prompt(question, df, max_prompt_tokens)

    try:
        response = openai.Completion.create(
            model=COMPLETION_MODEL_NAME,
            prompt=prompt,
            max_tokens=max_answer_tokens
        )
        return response["choices"][0]["text"].strip()
    except Exception as e:
        print(e)
        return ""

NameError: name 'get_rows_sorted_by_relevance' is not defined

## Custom Performance Demonstration

TODO: In the cells below, demonstrate the performance of your custom query using at least 2 questions. For each question, show the answer from a basic `Completion` model query as well as the answer from your custom query.

### Question 1

In [ ]:
question = "Who was the president of the United States at the beginning of 2024?"
answer = answer_question(question, df)

### Question 2